In [3]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [5]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = ""
)

ModuleNotFoundError: No module named 'unsloth'

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

In [7]:
import pandas as pd

df = pd.read_csv('/content/cti-rcm.tsv', sep='\t')

# Randomly sample 100 rows, ensuring no missing values in 'Description' and 'GT'
df_sample = df.sample(n=100, random_state=42).dropna(subset=['Description', 'GT']).reset_index(drop=True)

print(f"Sampled {len(df_sample)} entries for evaluation.")



Sampled 100 entries for evaluation.


In [8]:
# Function to generate CWE ID using the LLaMA model
def generate_cwe_id(description):
    prompt = f"""
    Analyze the following CVE description and map it to the appropriate CWE.
    Provide a brief justification for your choice.
    Ensure the last line of your response contains only the CWE ID.
    CVE Description: {description}
    """

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_seq_length).to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    print(f"Model Response:\n{response}")  # Debugging print

    # Extract the last line as the CWE ID
    return response.split('\n')[-1]



In [9]:
# Apply the model to generate CWE IDs for each description in the sample
df_sample['Generated_CWE_ID'] = df_sample['Description'].apply(generate_cwe_id)



Model Response:
Analyze the following CVE description and map it to the appropriate CWE.
    Provide a brief justification for your choice.
    Ensure the last line of your response contains only the CWE ID.
    CVE Description: A SQL injection vulnerability exists in Novel-Plus v4.3.0-RC1 and prior. An attacker can pass specially crafted offset, limit, and sort parameters to perform SQL injection via /novel/userFeedback/list.
     CWE ID: CWE-89 (SQL Injection) is the appropriate CWE for this vulnerability. The attacker is able to inject malicious SQL code into the application's database, which can lead to data disclosure or modification. This is a common vulnerability in web applications that use SQL databases.
Model Response:
Analyze the following CVE description and map it to the appropriate CWE.
    Provide a brief justification for your choice.
    Ensure the last line of your response contains only the CWE ID.
    CVE Description: Due to a failure in validating the number of sca

In [10]:
# Save the intermediate results for later use
df_sample.to_csv('cve_predictions.tsv', sep='\t', index=False)


In [11]:
# Function to compute RCM accuracy from the paper
def compute_rcm_accuracy(fname, col):
    """Compute RCM accuracy by comparing predicted CWE with ground truth."""
    df = pd.read_csv(fname, sep='\t')
    correct = 0
    total = 0

    for idx, row in df.iterrows():
        pred = row[col].upper()
        gt = row['GT'].upper()

        # Validate if the prediction starts with 'CWE-'
        if pred.startswith('CWE-'):
            total += 1
            if pred == gt:
                correct += 1
        else:
            print(f'Invalid response at row {idx + 1}')

    # Calculate and return the accuracy
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy


In [12]:

# Compute the accuracy using the saved predictions
accuracy = compute_rcm_accuracy('/content/cve_predictions.tsv', 'Generated_CWE_ID')
print(f"Final RCM Accuracy: {accuracy:.2f}%")

Invalid response at row 1
Invalid response at row 2
Invalid response at row 3
Invalid response at row 4
Invalid response at row 5
Invalid response at row 6
Invalid response at row 7
Invalid response at row 8
Invalid response at row 9
Invalid response at row 10
Invalid response at row 11
Invalid response at row 12
Invalid response at row 13
Invalid response at row 14
Invalid response at row 15
Invalid response at row 16
Invalid response at row 17
Invalid response at row 18
Invalid response at row 19
Invalid response at row 20
Invalid response at row 21
Invalid response at row 22
Invalid response at row 23
Invalid response at row 24
Invalid response at row 25
Invalid response at row 26
Invalid response at row 27
Invalid response at row 28
Invalid response at row 29
Invalid response at row 30
Invalid response at row 31
Invalid response at row 32
Invalid response at row 33
Invalid response at row 34
Invalid response at row 35
Invalid response at row 36
Invalid response at row 37
Invalid re

In [14]:
d=pd.read_csv('/content/cve_predictions.tsv', sep='\t')
d

,URL,Description,Prompt,GT,Generated_CWE_ID
0,https://nvd.nist.gov/vuln/detail/CVE-2024-24021,A SQL injection vulnerability exists in Novel-...,Analyze the following CVE description and map ...,CWE-89,CWE ID: CWE-89 (SQL Injection) is the app...
1,https://nvd.nist.gov/vuln/detail/CVE-2023-5841,Due to a failure in validating the number of s...,Analyze the following CVE description and map ...,CWE-787,CVE Description: Due to a failure in valid...
2,https://nvd.nist.gov/vuln/detail/CVE-2023-51678,Cross-Site Request Forgery (CSRF) vulnerabilit...,Analyze the following CVE description and map ...,CWE-352,Last line:
3,https://nvd.nist.gov/vuln/detail/CVE-2023-4960,The WCFM Marketplace plugin for WordPress is v...,Analyze the following CVE description and map ...,CWE-79,CVE Description: The WCFM Marketplace plug...
4,https://nvd.nist.gov/vuln/detail/CVE-2023-51969,Tenda AX1803 v1.0.0.1 contains a stack overflo...,Analyze the following CVE description and map ...,CWE-787,- The stack overflow can be exploited to ...
...,...,...,...,...,...
95,https://nvd.nist.gov/vuln/detail/CVE-2023-51722,This vulnerability exist in Skyworth Router CM...,Analyze the following CVE description and map ...,CWE-79,Justification: This vulnerability exists ...
96,https://nvd.nist.gov/vuln/detail/CVE-2024-0576,A vulnerability was found in Totolink LR1200GB...,Analyze the following CVE description and map ...,CWE-121,CVE Description: A vulnerability was found...
97,https://nvd.nist.gov/vuln/detail/CVE-2023-7029,The WordPress Button Plugin MaxButtons plugin ...,Analyze the following CVE description and map ...,CWE-79,Justification:
98,https://nvd.nist.gov/vuln/detail/CVE-2024-26909,"In the Linux kernel, the following vulnerabili...",Analyze the following CVE description and map ...,CWE-416,"CVE Description: In the Linux kernel, the ..."
